In [9]:
import torch
from torch import nn
import torch.nn.functional as F

In [10]:
a = torch.ones(4, requires_grad=True)
b = torch.ones(4, requires_grad=True)
w = a.mean()
y = b.mean()
z = a.mean() + b.mean().detach()
opt1 = torch.optim.SGD([a], lr=1)
opt2 = torch.optim.SGD([b], lr=1)
opt3 = torch.optim.SGD([a, b], lr=1)
w.backward()
print('Now the gradients of a is {}'.format(a.grad))
opt2.step()
print('Now the value of a is {}, b is {}'.format(a, b))
y.backward()
print('Now the gradients of a is {}, b is {}'.format(a.grad, b.grad))
z.backward()
print('Now the gradients of a is {}, b is {}'.format(a.grad, b.grad))
opt3.step()
print('Now the value of a is {}, b is {}'.format(a, b))

Now the gradients of a is tensor([ 0.2500,  0.2500,  0.2500,  0.2500])
Now the value of a is tensor([ 1.,  1.,  1.,  1.]), b is tensor([ 1.,  1.,  1.,  1.])
Now the gradients of a is tensor([ 0.2500,  0.2500,  0.2500,  0.2500]), b is tensor([ 0.2500,  0.2500,  0.2500,  0.2500])
Now the gradients of a is tensor([ 0.5000,  0.5000,  0.5000,  0.5000]), b is tensor([ 0.2500,  0.2500,  0.2500,  0.2500])
Now the value of a is tensor([ 0.5000,  0.5000,  0.5000,  0.5000]), b is tensor([ 0.7500,  0.7500,  0.7500,  0.7500])


In [148]:
net = nn.Linear(4,1)
opt = torch.optim.SGD(net.parameters(), lr=1)
a = torch.ones(4)
z = net(a)
z.backward()
print(net.weight.grad)
print(net.weight.data)
opt.step()
print(net.weight.data)
opt.step()
print(net.weight.data)
opt2 = torch.optim.SGD(net.parameters(), lr=1)
opt2.step()
print(net.weight.data)
opt2.zer

tensor([[ 1.,  1.,  1.,  1.]])
tensor([[ 0.1665,  0.4128, -0.0153,  0.1954]])
tensor([[-0.8335, -0.5872, -1.0153, -0.8046]])
tensor([[-1.8335, -1.5872, -2.0153, -1.8046]])
tensor([[-2.8335, -2.5872, -3.0153, -2.8046]])


试验一下backward之后节点还在不在

In [105]:
net = nn.Linear(4,1)
a = torch.ones(4)
z = net(a)
z.backward()
print(z) #发现z还在 只不过不能再次求导

tensor([ 0.2553])


对于同一个graph, 如果两次backward的路径不交叉，则可以backward两次

In [110]:
a = torch.ones(4)
b = torch.ones(4)
net1 = nn.Linear(4,1)
net2 = nn.Linear(4,1)
u = net(a)
v = net(b)
y = u + v.detach()
z = u.detach() + v
y.backward()
z.backward()

但是如果两次求导的路径是交叉的，则不可以backward两次

In [111]:
a = torch.ones(4)
b = torch.ones(4)
net1 = nn.Linear(4,1)
net2 = nn.Linear(4,1)
u = net(a)
v = net(b)
y = u + v.detach()
z = u + v
y.backward()
z.backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

探求retain_graph=True的情形,使用retain_graph=True 即使在两次backward的过程之间使用了opt.step()导致net的参数改变，第二次backward还是基于net没改变之前计算导数

In [139]:
net = nn.Linear(4,1)
a = torch.ones(4)
w = net(a) * net(a)
print(2 * net(a))
opt = torch.optim.SGD(net.parameters(), lr=0.01)
w.backward(retain_graph=True)
for param in net.parameters():
    print(param.grad)

tensor([ 4.2079])
tensor([[ 4.2079,  4.2079,  4.2079,  4.2079]])
tensor([ 4.2079])


In [140]:
opt.step()
opt.zero_grad()

In [141]:
w.backward()
print(2 * net(a))
for param in net.parameters():
    print(param.grad)

tensor([ 3.7871])
tensor([[ 4.2079,  4.2079,  4.2079,  4.2079]])
tensor([ 4.2079])


In [142]:
opt.zero_grad()
a = torch.ones(4, requires_grad=True)
w = net(a) * net(a)
print(2 * net(a))
w.backward(retain_graph=True)
for param in net.parameters():
    print(param.grad)

tensor([ 3.7871])
tensor([[ 3.7871,  3.7871,  3.7871,  3.7871]])
tensor([ 3.7871])


In [143]:
opt.step()
opt.zero_grad()
print(2 * net(a))
w.backward()
for param in net.parameters():
    print(param.grad)

tensor([ 3.4084])
tensor([[ 3.7871,  3.7871,  3.7871,  3.7871]])
tensor([ 3.7871])
